## Basic import

Basic import used in notebooks. Can be accessed by other notebooks.

In [ ]:
#These are used in this ipynb and others.
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from fitparse import FitFile
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

#These are used in other ipynb.
import matplotlib.pyplot as plt

In [ ]:
location_string = '../../../fitfiles/'
participant_files = []
for f in listdir(location_string):
    if isfile(location_string+f):
        participant_files.append(f)

In [ ]:
def speed_corr(speed):
    return speed if isinstance(speed, float) else 0

def power_corr(power):
    return power if isinstance(power, int) else 0
    
def read_record(rec):
    global curr_timestamp
    global curr_stop
    if curr_stop == 1 and speed_corr(rec.get_value('speed')) > 7:
        curr_stop = 0
    curr_timestamp = rec.get_value('timestamp')
    return (curr_timestamp,
        1, #type = 1
        curr_lap,
        curr_stop, 
        rec.get_value('speed'), 
        rec.get_value('enhanced_speed'),
        rec.get_value('distance'), 
        np.NaN, #HRV
        rec.get_value('heart_rate'), 
        rec.get_value('accumulated_power'), 
        rec.get_value('power'), 
        rec.get_value('position_lat'), 
        rec.get_value('position_long'), 
        rec.get_value('altitude'), 
        rec.get_value('enhanced_altitude'), 
        rec.get_value('cadence'), 
        rec.get_value('factional_cadence'), 
        rec.get_value('left_right_balnce'), 
        rec.get_value('left_torque_effectiveness'), 
        rec.get_value('right_torque_effectiveness'), 
        rec.get_value('left_pedal_smoothness'), 
        rec.get_value('right_pedal_smoothness'), 
        rec.get_value('temperature'))

def read_lap(rec):
    global curr_timestamp
    global curr_lap
    global curr_stop
    curr_lap += 1
    curr_stop = 1
    curr_timestamp = rec.get_value('timestamp')
    return (curr_timestamp, 
            2, # type = 2
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            rec.get_value('total_distance'),
            np.NaN, #HRV 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

def read_hrv(rec):
    hrv = []
    hrv_timestamps = []
    for i in range(len(rec.get_value('time'))):
        if rec.get_value('time')[i] != None:
            hrv.append(rec.get_value('time')[i])
    for i in range(len(hrv)):
        if i != 0:
            hrv[i] = hrv[i] + hrv[i-1]
    for i in range(len(hrv)):
        hrv_timestamps.append(curr_timestamp+pd.to_timedelta(hrv[i],unit='S'))
    return (curr_timestamp, 
            3, # type = 2
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            np.NaN,
            hrv_timestamps, #HRV 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

In [ ]:
def fitgenerator(fitfile):
    for rec in fitfile.get_messages(['record', 'lap', 'hrv']):
        if rec.name=='record':
            yield read_record(rec)
        elif rec.name=='hrv':
            yield read_hrv(rec)
        else:
            yield read_lap(rec)

In [ ]:
fitgen_cols = ['timestamp', 'type', 'lap', 'stop', 'speed', 'enhanced_speed', 'distance', 'hrv',                           'heart_rate', 
               'accumulated_power', 'power', 'position_lat', 'position_long', 
               'altitude', 'enhanced_altitude', 'cadence', 'factional_cadence', 
               'left_right_balnce', 'left_torque_effectiveness', 'right_torque_effectiveness', 
               'left_pedal_smoothness', 'right_pedal_smoothness', 'temperature']
all_dfs = []
all_laps = []
all_hrv = []
all_messages = []
#dStops is -30 +30 seconds.
all_dStops = []
all_stops = []

In [ ]:
for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)
    
    curr_lap = 0
    curr_stop = 0
    #curr_active = 0

    fitgen = fitgenerator(fitfile)
    part_df = pd.DataFrame(fitgen, columns=fitgen_cols)
    all_dfs.append(part_df[part_df['type']==1])
    all_stops.append(part_df[part_df['type']==2].set_index('timestamp'))
    all_hrv.append(part_df[part_df['type']==3])
    all_messages.append(part_df)